<a href="https://www.kaggle.com/code/jaswanth431/dl-assignement-3-v2?scriptVersionId=175612901" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [34]:
import torch
from torch import nn
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import copy
from torch.utils.data import Dataset, DataLoader
import gc
import random

In [35]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
#Global constants
MAX_LENGTH = 30
INPUT_MAX_LENGTH = 0
OUTPUT_MAX_LENGTH = 0
END_TOKEN = '>'
START_TOKEN = '<'
PAD_TOKEN = '_'
TEACHER_FORCING_RATIO = 0.5

cuda


In [36]:
# train_csv = "/kaggle/input/aksh11/aksharantar_sampled/tel/tel_train.csv"
# test_csv = "/kaggle/input/aksh11/aksharantar_sampled/tel/tel_test.csv"
# val_csv = "/kaggle/input/aksh11/aksharantar_sampled/tel/tel_valid.csv"
train_csv = "/kaggle/input/aksh11/aksharantar_sampled/hin/hin_train.csv"
test_csv = "/kaggle/input/aksh11/aksharantar_sampled/hin/hin_test.csv"
val_csv = "/kaggle/input/aksh11/aksharantar_sampled/hin/hin_valid.csv"

In [37]:
train_df = pd.read_csv(train_csv, header=None)
train_source, train_target = train_df[0].to_numpy(), train_df[1].to_numpy();
test_df = pd.read_csv(test_csv, header=None)
val_df = pd.read_csv(val_csv, header=None)
val_source, val_target = val_df[0].to_numpy(), val_df[1].to_numpy();


In [38]:
def add_padding(source_data, MAX_LENGTH):
    padded_source_strings = []
    for i in range(len(source_data)):
        source_str =START_TOKEN+ source_data[i] + END_TOKEN
        # Truncate or pad source sequence
        source_str = source_str[:MAX_LENGTH]
        source_str += PAD_TOKEN * (MAX_LENGTH - len(source_str))

        padded_source_strings.append(source_str)
        
    return padded_source_strings


def generate_string_to_sequence(source_data, source_char_index_dict):
    source_sequences = []
    for i in range(len(source_data)):
        source_sequences.append(get_chars(source_data[i], source_char_index_dict))
    source_sequences = pad_sequence(source_sequences, batch_first=True, padding_value=2)
    return source_sequences


def get_chars(str, char_index_dict):
    chars_indexes = []
    for ch in str:
        chars_indexes.append(char_index_dict[ch])
    return torch.tensor(chars_indexes, device=device)


def preprocess_data(source_data, target_data):
    data = {
        "source_chars": [START_TOKEN, END_TOKEN, PAD_TOKEN],
        "target_chars": [START_TOKEN, END_TOKEN, PAD_TOKEN],
        "source_char_index": {START_TOKEN: 0, END_TOKEN:1, PAD_TOKEN:2},
        "source_index_char": {0:START_TOKEN, 1: END_TOKEN, 2:PAD_TOKEN},
        "target_char_index": {START_TOKEN: 0, END_TOKEN:1, PAD_TOKEN:2},
        "target_index_char": {0:START_TOKEN, 1: END_TOKEN, 2:PAD_TOKEN},
        "source_len": 3,
        "target_len": 3,
        "source_data": source_data,
        "target_data": target_data,
        "source_data_seq": [],
        "target_data_seq": []
    }
    
    data["INPUT_MAX_LENGTH"] = max(len(string) for string in source_data) +2
    data["OUTPUT_MAX_LENGTH"] = max(len(string) for string in target_data)+2

    
    padded_source_strings=add_padding(source_data, data["INPUT_MAX_LENGTH"])
    padded_target_strings = add_padding(target_data, data["OUTPUT_MAX_LENGTH"])
    
    for i in range(len(padded_source_strings)):
        for c in padded_source_strings[i]:
            if data["source_char_index"].get(c) is None:
                data["source_chars"].append(c)
                idx = len(data["source_chars"]) - 1
                data["source_char_index"][c] = idx
                data["source_index_char"][idx] = c
        for c in padded_target_strings[i]:
            if data["target_char_index"].get(c) is None:
                data["target_chars"].append(c)
                idx = len(data["target_chars"]) - 1
                data["target_char_index"][c] = idx
                data["target_index_char"][idx] = c

    data['source_data_seq'] = generate_string_to_sequence(padded_source_strings,  data['source_char_index'])
    data['target_data_seq'] = generate_string_to_sequence(padded_target_strings,  data['target_char_index'])
    print(data["source_data"][0])
    print(data["source_data_seq"][0])
    print(data["target_data"][0])
    print(data["target_data_seq"][0])

    
    data["source_len"] = len(data["source_chars"])
    data["target_len"] = len(data["target_chars"])
    
    return data


data = preprocess_data(copy.copy(train_source), copy.copy(train_target[:]))
print(data["source_chars"])
print(data["target_chars"])
print(data["source_len"])
print(data["target_len"])
print(data["INPUT_MAX_LENGTH"])
print(data["OUTPUT_MAX_LENGTH"])



shastragaar
tensor([0, 3, 4, 5, 3, 6, 7, 5, 8, 5, 5, 7, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2], device='cuda:0')
शस्त्रागार
tensor([0, 3, 4, 5, 6, 5, 7, 8, 9, 8, 7, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       device='cuda:0')
['<', '>', '_', 's', 'h', 'a', 't', 'r', 'g', 'b', 'i', 'n', 'd', 'y', 'k', 'o', 'p', 'v', 'e', 'c', 'm', 'u', 'w', 'l', 'j', 'x', 'f', 'z', 'q']
['<', '>', '_', 'श', 'स', '्', 'त', 'र', 'ा', 'ग', 'ब', 'ि', 'न', 'द', 'य', 'क', 'ण', 'ं', 'ज', 'ञ', 'ो', 'प', 'व', 'ी', 'ट', 'च', 'े', 'भ', 'म', 'ध', 'ु', 'घ', 'ड', '़', 'ह', 'ल', 'ै', 'इ', 'ॉ', 'ू', 'अ', 'ए', 'ौ', 'आ', 'ई', 'झ', 'ः', 'ख', 'ष', 'उ', 'थ', 'छ', 'ठ', 'ँ', 'ओ', 'फ', 'ढ', 'ऊ', 'ृ', 'ऐ', 'ळ', 'ऋ', 'औ', 'ऑ', 'ॅ', 'ङ', 'ऽ']
29
67
22
26


In [39]:
h_params={
    "char_embd_dim" : 256, 
    "hidden_layer_neurons":512,
    "batch_size":64,
    "number_of_layers":3,
    "learning_rate":0.001,
    "epochs":30,
    "cell_type":"LSTM",
    "dropout":0,
    "optimizer":"adam"
}

In [40]:
def get_cell_type(cell_type):
    if(cell_type == "RNN"):
        return nn.RNN
    elif(cell_type == "LSTM"):
        return nn.LSTM
    elif(cell_type == "GRU"):
        return nn.GRU
    else:
        print("Specify correct cell type")
        
class Encoder(nn.Module):
    def __init__(self, h_params, data, device ):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(data["source_len"], h_params["char_embd_dim"])
        self.dropout = nn.Dropout(h_params["dropout"])
        self.cell = get_cell_type(h_params["cell_type"])(h_params["char_embd_dim"], h_params["hidden_layer_neurons"],num_layers=h_params["number_of_layers"], dropout= h_params["dropout"],batch_first=True)
        self.device=device
        self.h_params = h_params
    def forward(self, current_input, prev_state):
        embd_input = self.embedding(current_input)
        embd_input = self.dropout(embd_input)
        output, prev_state = self.cell(embd_input, prev_state)
        return output, prev_state
    
    def getInitialState(self):
        return torch.zeros(h_params["number_of_layers"],h_params["batch_size"],h_params["hidden_layer_neurons"], device=self.device)

    
class Decoder(nn.Module):
    def __init__(self, h_params, data,device):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(data["target_len"], h_params["char_embd_dim"])
        self.dropout = nn.Dropout(h_params["dropout"])
        self.cell = get_cell_type(h_params["cell_type"])(h_params["char_embd_dim"], h_params["hidden_layer_neurons"],num_layers=h_params["number_of_layers"],dropout= h_params["dropout"], batch_first=True)
        self.fc = nn.Linear(h_params["hidden_layer_neurons"], data["target_len"])
        self.softmax = nn.LogSoftmax(dim=2)
        self.h_params = h_params

    def forward(self, current_input, prev_state):
        embd_input = self.embedding(current_input)
        curr_embd = F.relu(embd_input)
        curr_embd = self.dropout(curr_embd)
        output, prev_state = self.cell(curr_embd, prev_state)
        output = self.softmax(self.fc(output))
        return output, prev_state

In [41]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.source_data_seq = data[0]
        self.target_data_seq = data[1]
    
    def __len__(self):
        return len(self.source_data_seq)
    
    def __getitem__(self, idx):
        source_data = self.source_data_seq[idx]
        target_data = self.target_data_seq[idx]
        return source_data, target_data

In [42]:
val_padded_source_strings=add_padding(val_source, data["INPUT_MAX_LENGTH"])
val_padded_target_strings = add_padding(val_target, data["OUTPUT_MAX_LENGTH"])
val_source_sequences = generate_string_to_sequence(val_padded_source_strings,  data['source_char_index'])
val_target_sequences = generate_string_to_sequence(val_padded_target_strings,  data['target_char_index'])
validation_data = [val_source_sequences, val_target_sequences]
val_dataset = MyDataset(validation_data)
val_dataloader = DataLoader(val_dataset, batch_size=h_params["batch_size"], shuffle=True)

In [43]:
def inference(encoder, decoder, source_sequence, target_tensor, data, device, h_params, loss_fn, batch_num):
    encoder.eval()
    decoder.eval()
    
    loss = 0
    correct = 0
    with torch.no_grad():
        encoder_hidden = encoder.getInitialState()
#         print(source_sequence.shape)
#         print(source_sequence)
        if h_params["cell_type"] == "LSTM":
            encoder_hidden = (encoder_hidden, encoder.getInitialState())
        encoder_outputs, encoder_hidden = encoder(source_sequence, encoder_hidden)
#         print(encoder_hidden.shape)
#         print(encoder_hidden)

        decoder_input = torch.full((h_params["batch_size"], 1), data['target_char_index'][START_TOKEN], device=device)  # Start token
#         print(decoder_input)
        decoder_actual_output = []
        
        decoder_hidden = encoder_hidden
        for di in range(data["OUTPUT_MAX_LENGTH"]):
            curr_target_chars = target_tensor[:, di]
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input_tensor = topi.squeeze().detach()
#             print(decoder_input_tensor.shape)
#             print(decoder_input_tensor)
            decoder_actual_output.append(decoder_input_tensor)
            decoder_input_tensor = decoder_input_tensor.view(h_params["batch_size"], 1)
                        
            decoder_output = decoder_output[:, -1, :]
            loss+=(loss_fn(decoder_output, curr_target_chars))

        decoder_actual_output = torch.cat(decoder_actual_output,dim=0).view(data["OUTPUT_MAX_LENGTH"], h_params["batch_size"]).transpose(0,1)
        if(batch_num %100== 0):
                print("<<<<<<<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\n")
                for j in range(20):
                    print("batch num: ", batch_num, "val_acc", make_strings(data,source_sequence[j],target_tensor[j],decoder_actual_output[j]))

        correct = (decoder_actual_output == target_tensor).all(dim=1).sum().item()
#         if(batch_num%20 == 0):
#             print("ep:", ep, " bt:", batch_num, " loss:", loss.item()/output_seq_len, " acc: ", correct/h_params["batch_size"])
        return correct, loss.item()/data["OUTPUT_MAX_LENGTH"]
    
    
def evaluate(encoder, decoder, data, dataloader, device, h_params, loss_fn):
    correct_predictions = 0
    total_loss = 0
    total_predictions = len(dataloader.dataset)
    number_of_batches = len(dataloader)
    print(number_of_batches)
    print(total_predictions)
    for batch_num, (source_sequence, target_sequence) in enumerate(dataloader):
        input_tensor = source_sequence
        target_tensor = target_sequence
        
        correct, loss = inference(encoder, decoder, input_tensor, target_tensor, data, device, h_params, loss_fn, batch_num)
        
        correct_predictions+=correct
        total_loss +=loss
    
    accuracy = correct_predictions / total_predictions
    total_loss /= number_of_batches
    
    print("accuracy: ", accuracy, " loss:", total_loss)
    return accuracy, total_loss


In [44]:
def make_strings(data, source, target, output):
    source_string = ""
    target_string = ""
    output_string = ""
#     print(output)
    for i in source:
#         print(i.item())
        source_string+=(data['source_index_char'][i.item()])
    for i in target:
        target_string+=(data['target_index_char'][i.item()])
    for i in output:
        output_string+=(data['target_index_char'][i.item()])
    return source_string, target_string, output_string



def train_loop(encoder, decoder,h_params, data, data_loader, device):
    if h_params["optimizer"] == "adam":
        encoder_optimizer = optim.Adam(encoder.parameters(), lr=h_params["learning_rate"])
        decoder_optimizer = optim.Adam(decoder.parameters(), lr=h_params["learning_rate"])
    elif h_params["optimizer"] == "nadam":
        encoder_optimizer = optim.NAdam(encoder.parameters(), lr=h_params["learning_rate"])
        decoder_optimizer = optim.NAdam(decoder.parameters(), lr=h_params["learning_rate"])
    
    total_predictions = len(data_loader.dataset)
    total_batches = len(data_loader)
    loss_fn = nn.NLLLoss()
    for ep in range(h_params["epochs"]):
        encoder.train()
        decoder.train()
        total_loss = 0
        total_correct = 0
        for batch_num, (source_batch, target_batch) in enumerate(data_loader):
#             if(batch_num>0):
#                 break
            encoder_initial_state = encoder.getInitialState()
            if h_params["cell_type"] == "LSTM":
                encoder_initial_state = (encoder_initial_state, encoder.getInitialState())
            
            encoder_current_state = encoder_initial_state
            encoder_output, encoder_current_state = encoder(source_batch,encoder_current_state)
            loss = 0
            correct = 0
            decoder_curr_state = encoder_current_state
            output_seq_len = data["OUTPUT_MAX_LENGTH"]
            decoder_actual_output = []
            
            use_teacher_forcing = True if random.random() < TEACHER_FORCING_RATIO else False

            for i in range(data["OUTPUT_MAX_LENGTH"]):
                if(i == 0):
                    dec_input_tensor = target_batch[:, i].view(h_params["batch_size"],1)
                curr_target_chars = target_batch[:, i]
                decoder_output, decoder_curr_state = decoder(dec_input_tensor,decoder_curr_state)
                topv, topi = decoder_output.topk(1)
                decoder_input_tensor = topi.squeeze().detach()
                decoder_actual_output.append(decoder_input_tensor)
                if(i<output_seq_len-1):
                    if use_teacher_forcing:
                        decoder_input_tensor = target_batch[:, i+1].view(h_params["batch_size"], 1)
                    else:
                        decoder_input_tensor = decoder_input_tensor.view(h_params["batch_size"], 1)
                decoder_output = decoder_output[:, -1, :]
                loss+=(loss_fn(decoder_output, curr_target_chars))

            decoder_actual_output = torch.cat(decoder_actual_output,dim=0).view(output_seq_len, h_params["batch_size"]).transpose(0,1)
            if(batch_num == 0):
                    for j in range(20):
                        print(make_strings(data,source_batch[j],target_batch[j],decoder_actual_output[j]))
            
            correct = (decoder_actual_output == target_batch).all(dim=1).sum().item()
            total_correct+=correct;
            total_loss += loss.item()/output_seq_len;
            if(batch_num%50 == 0):
                print("ep:", ep, " bt:", batch_num, " loss:", loss.item()/output_seq_len, " acc: ", correct/h_params["batch_size"])
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
        
        print("ep: ", ep, " train acc:", total_correct/total_predictions, " loss:", total_loss/total_batches)
        evaluate(encoder, decoder, data, val_dataloader,device, h_params, loss_fn)
#     encoder=None
#     decoder=None
#     gc.collect()
#     torch.cuda.empty_cache() 
    return encoder, decoder, loss_fn

In [ ]:
training_data = [data["source_data_seq"], data['target_data_seq']]
my_dataset = MyDataset(training_data)
data_loader = DataLoader(my_dataset, batch_size=h_params["batch_size"], shuffle=True)
def train(h_params, data, device, data_loader):
    encoder = Encoder(h_params, data, device).to(device)
    decoder = Decoder(h_params, data, device).to(device)
    
    encoder,  decoder, loss_fn = train_loop(encoder, decoder,h_params, data, data_loader,device)
    return encoder, decoder, loss_fn

encoder, decoder, loss_fn = train(h_params, data, device, data_loader)


('<auratamaam>______________', '<औरतमाम>______________', 'ञाससणााााााााााााााााा')
('<otgun>___________________', '<ओटगन>________________', 'ञाससणााााााााााााााााा')
('<krishnamurari>___________', '<कृष्णमुरारी>_________', 'ञाससणााााााााााााााााा')
('<dhyanbal>________________', '<ध्यानबल>_____________', 'ञाससणााााााााााााााााा')
('<tikarthiyon>_____________', '<टिकार्थियों>_________', 'ञाससणााााााााााााााााा')
('<rangilpura>______________', '<रंगीलपुरा>___________', 'ञाससणााााााााााााााााा')
('<chupchupa>_______________', '<चुपचुपा>_____________', 'ञाससणााााााााााााााााा')
('<scalaper>________________', '<स्केलेपर>____________', 'ञाससणााााााााााााााााा')
('<singhohi>________________', '<सिंघोही>_____________', 'ञाससणााााााााााााााााा')
('<duhariya>________________', '<दुहरिया>_____________', 'ञाससणााााााााााााााााा')
('<ranimandi>_______________', '<रानीमंडी>____________', 'ञाससणााााााााााााााााा')
('<quwatul>_________________', '<कुबातुल>_____________', 'ञाससणााााााााााााााााा')
('<r

In [ ]:
#Run this cell to train a model for a h_params configuration
config = h_params
training_data = prepare_data(config)
run = wandb.init(project="DL Assignment 2", name=f"{config['actv_func']}_ep_{config['epochs']}_lr_{config['learning_rate']}_init_fltr_cnt_{config['num_of_filter']}_fltr_sz_{config['filter_size']}_fltr_mult_{config['filter_multiplier']}_data_aug_{config['data_augumentation']}_batch_norm_{config['batch_normalization']}_dropout_{config['dropout']}_dense_size_{config['dense_layer_size']}", config=config)
train(config, training_data) 

In [ ]:
#Run this cell to run a sweep with appropriate parameters
sweep_params = {
    'method' : 'bayes',
    'name'   : 'DL assn 2 sweep',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'val_accuracy',
    },
    'parameters' : {
        'epochs':{'values' : [10]},
        'learning_rate':{'values' : [0.0001, 0.001]},
        'batch_size':{'values':[32,64]},
        'num_of_filter':{'values' : [16,32,64] } ,
        'filter_size':{'values' : [[3,3,3,3,3], [5,5,5,5,5], [7,7,7,7,7], [11,9,7,5,3], [3,5,7,9,11]]},
        'actv_func':{'values':['elu','gelu','leaky_relu','selu']},
        'filter_multiplier':{'values' : [1,2]},
        'data_augumentation':{'values': [ False]},
        'batch_normalization':{'values' : [True, False]},
        'dropout':{'values': [0,0.1,0.2]},
        'dense_layer_size':{'values' : [64, 128,256]},
        'conv_layers':{'values':[5]}
    }
}

sweep_id = wandb.sweep(sweep=sweep_params, project="DL Assignment 2")
def main():
    wandb.init(project="DL Assignment 2" )
    config = wandb.config
    with wandb.init(project="DL Assignment 2", name=f"{config['actv_func']}_ep_{config['epochs']}_lr_{config['learning_rate']}_init_fltr_cnt_{config['num_of_filter']}_fltr_sz_{config['filter_size']}_fltr_mult_{config['filter_multiplier']}_data_aug_{config['data_augumentation']}_batch_norm_{config['batch_normalization']}_dropout_{config['dropout']}_dense_size_{config['dense_layer_size']}_batch_size_{config['batch_size']}", config=config ):
        training_data = prepare_data(config)  
        train(config,training_data)
wandb.agent(sweep_id, function=main, count=10)